# Monads for normal people!

Dustin Getz
@dustingetz


# intended audience

* coders

* who are comfortable with lambdas

* who learn by example

# goals

* how monads work?
* how do monads help?
* are monads useful IRL?
* especially in enterprise?
* where do they fall short and what's next?

# large codebases are complex

* Spring, EJB, AspectJ, DI, AOP
* Common goal: make code look like business logic
* (to varying degrees of success)

# Aspect Oriented Programming

*From Wikipedia...*

# Lots of code to follow

* Pay attention to how functions compose

# a bank API

In [1]:
from collections import namedtuple

Person = namedtuple('Person', 'name')
Account = namedtuple('Account', 'id')
Balance = namedtuple('Balance', ['cash', 'ccy'])

alice = Person('Alice')
bob = Person('Bob')


def get_account(person):
    if person.name == 'Alice': return Account(1)
    elif person.name == 'Bob': return Account(2)
    else: return None

def get_balance(account):
    if account.id == 1: return Balance(1000000, 'usd')
    elif account.id == 2: return Balance(75000, 'usd')
    else: return None

def get_qualified_amount(balance):
    if balance.cash > 200000: return balance.cash
    else: return None


# what we want to write

In [2]:
def get_loan(name):
    account = get_balance(name)
    balance = get_balance(account)
    loan = get_qualified_amount(balance)
    return loan

# boss would write this code

```python
# POSIX

alice | get_account | get_balance | get_qualified_amount
```

In [3]:
# Functions

get_qualified_amount( get_balance( get_account( alice ) ) )

1000000

```python
# Object-Oriented

alice.get_account().get_balance().get_qualified_amount()
```

# I love `AttributeErrors` !

In [4]:
# get_account(None)

# what the production code looks like :-(

In [5]:
def get_loan(name):
    account = get_balance(name)
    if not account:
        return None

    balance = get_balance(account)
    if not balance:
        return None

    loan = get_qualified_amount(balance)
    return loan

# factor! abstract! happy!

In [6]:
def bind(v, f):
    if v:            # v == alice
        return f(v)  # get_account(alice)
    else:
        return None


In [8]:
alice = Person('Alice')
bind(alice, get_account)

Account(id=1)

In [ ]:
def bind(v, f):
    if v:
        return f(v)
    else:            # v == None
        return None  # don't call f


In [11]:
alice = Person('Alice')
print(bind(None, get_account))

None


# factor! abstract! happy!